In [1]:
import numpy as np
import veloxchem as vlx

mol_str = """2

C        -0.71500000    0.00000000   0.00000000
O         0.71500000    0.00000000   0.00000000
"""
molecule = vlx.Molecule.read_xyz_string(mol_str)
basis = vlx.MolecularBasis.read(molecule, "cc-pVDZ", ostream=None)

scf_drv = vlx.ScfRestrictedDriver()
scf_drv.ostream.mute()
scf_results = scf_drv.compute(molecule, basis)

In [2]:
molecule.show()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [3]:
nocc = molecule.number_of_alpha_electrons()
C = scf_results["C_alpha"]  # MO coefficient matrix

E_nuc = molecule.nuclear_repulsion_energy()
E_tot = scf_drv.get_scf_energy()
E_elec = E_tot - E_nuc

print("Number of occupied orbitals:", nocc)

print(f"Nuclear repulsion energy: {E_nuc : 19.12f}")
print(f"Electronic energy: {E_elec : 26.12f}")
print(f"Total Hartree-Fock energy: {E_tot : 18.12f}")

Number of occupied orbitals: 7
Nuclear repulsion energy:     17.762591694646
Electronic energy:          -130.366128020400
Total Hartree-Fock energy:  -112.603536325754


In [4]:
T_ao = vlx.compute_kinetic_energy_integrals(molecule, basis)
T = np.einsum("ai, ab, bj -> ij", C, T_ao, C)

V_ao = vlx.compute_nuclear_potential_integrals(molecule, basis)
V = np.einsum("ai, ab, bj -> ij", C, V_ao, C)

# core Hamiltonian
h = T + V

h_aver = 2 * np.einsum("ii", h[:nocc, :nocc])
print("Expectation value of one-electron Hamiltonian:", h_aver)

Expectation value of one-electron Hamiltonian: -188.97928532728596


In [5]:
# obtain the oooo-block
moints_drv = vlx.MOIntegralsDriver()
g_oooo = moints_drv.compute_in_memory(molecule, basis, scf_drv.mol_orbs, "chem_oooo")

g_aver = 2 * np.einsum("iijj", g_oooo) - np.einsum("ijji", g_oooo)
print("Expectation value of two-electron Hamiltonian:", g_aver)

Expectation value of two-electron Hamiltonian: 58.613157306885476


In [6]:
print(f"Expectation value of electronic Hamiltonian: {h_aver + g_aver:.12f}")

Expectation value of electronic Hamiltonian: -130.366128020400
